In [1]:
import pandas as pd
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 


In [2]:
train = pd.read_csv('./data/p_train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),precessed_price,index
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,NaN,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,NaN,1
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,0.0,1728.0,2
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,0.0,1408.0,3
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,893.4,1250.0,4


In [3]:
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))




In [4]:
import datetime
import holidays

# 공휴일 정보를 가져옴
holiday_list = holidays.Korea(years=datetime.datetime.now().year)

# train['holiday'] = 0

# # 특정 날짜가 공휴일인지 확인, 일요일 행은 train할때 다 지운다.
# for i in range(len(train)):
#     date = datetime.date(train.iloc[i,-4], train.iloc[i,-3], train.iloc[i,-2])
#     if date in holiday_list:
#         train.iloc[i,-1] = holiday_list[datetime.date(train.iloc[i, -4], train.iloc[i, -3], train.iloc[i, -2])] #공휴일이면 공휴일 이름 넣음

In [5]:
train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),precessed_price,index,Date,week,day_name,year,month,day
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,NaN,0,2019-01-01,1,Tuesday,2019,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,NaN,1,2019-01-02,1,Wednesday,2019,1,2
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,0.0,1728.0,2,2019-01-03,1,Thursday,2019,1,3
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,0.0,1408.0,3,2019-01-04,1,Friday,2019,1,4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,893.4,1250.0,4,2019-01-05,1,Saturday,2019,1,5


In [6]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# test_re['holiday'] = 0

# for i in range(len(test_re)):
#     date = datetime.date(test_re.iloc[i,5], test_re.iloc[i,6], test_re.iloc[i,7])
#     if date in holiday_list:
#         test_re.iloc[i,-1] = holiday_list[datetime.date(test_re.iloc[i, 5], test_re.iloc[i, 6], test_re.iloc[i, 7])] #공휴일이면 공휴일 이름 넣음

In [7]:
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [8]:
from sklearn.preprocessing import LabelEncoder

# 793점 방법
train_x_regression = train[train['day_name'] != 'Sunday'].copy()

train_x_regression = train_x_regression.drop(columns=['ID', 'timestamp', 'supply(kg)','Date', 'precessed_price'])

train_x_classification = train.drop(columns=['ID', 'timestamp', 'supply(kg)','Date'])

for k in range(len(train_x_classification)):
    if train_x_classification['price(원/kg)'][k] > 0:
        train_x_classification['price(원/kg)'][k] = 1

test_x = test_re.drop(columns=['ID', 'timestamp','Date'])


qual_col = ['item', 'corporation', 'location','day_name']
#del train_x['holiday']

#train_x = pd.get_dummies(train_x, columns=qual_col, prefix=qual_col)
#test_x = pd.get_dummies(test_x, columns=qual_col, prefix=qual_col)

for i in qual_col:
   le = LabelEncoder()
   train_x_classification[i]=le.fit_transform(train_x_classification[i])
   train_x_regression[i]=le.transform(train_x_regression[i]) # reg는 sunday지웠으니..

   test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다
    

# %%
#train_x_regression = pd.concat([train_x_regression, train_y], axis=1)
#train_x_classification = pd.concat([train_x_classification, train_y], axis =1)

In [9]:
from pycaret.classification import *

c = setup(train_x_classification, 
          target = 'price(원/kg)',
          fold = 10, 
          #numeric_features=train_x_classification.columns, 
          session_id = 123, 
          normalize = True, 
          normalize_method = 'minmax', 
          train_size=0.8)


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Binary
3,Original data shape,"(59397, 11)"
4,Transformed data shape,"(59397, 11)"
5,Transformed train set shape,"(47517, 11)"
6,Transformed test set shape,"(11880, 11)"
7,Numeric features,10
8,Rows with missing values,59.7%
9,Preprocess,True


In [10]:
m = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0190
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2120
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.5680
xgboost,Extreme Gradient Boosting,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.2170
catboost,CatBoost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.7970
rf,Random Forest Classifier,0.9999,1.0000,0.9998,1.0000,0.9999,0.9999,0.9999,0.2750
lightgbm,Light Gradient Boosting Machine,0.9999,0.9999,0.9998,1.0000,0.9999,0.9999,0.9999,0.1500
et,Extra Trees Classifier,0.9957,0.9999,0.9940,0.9954,0.9947,0.9911,0.9911,0.3150
qda,Quadratic Discriminant Analysis,0.9843,0.9927,0.9634,0.9976,0.9802,0.9673,0.9677,0.0210
nb,Naive Bayes,0.9836,0.9940,0.9616,0.9976,0.9793,0.9658,0.9662,0.0190


In [11]:
llar  = create_model('xgboost')
tuned_llar = tune_model(llar, optimize = 'F1')
predictions_classfication = predict_model(tuned_llar, data = train_x_classification)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [12]:
# 분류 결과 추가

In [13]:
train_df = pd.read_csv('./data/p_train.csv')[['ID','timestamp','supply(kg)', 'price(원/kg)', 'index']]
train_df['item_id'] = train_df.ID.str[0:6]

predictions_classfication_train_result = predict_model(llar, data = train_x_classification)
train_df['classies'] = predictions_classfication_train_result['prediction_label']
train_df['classies_score'] = predictions_classfication_train_result['prediction_score']

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [14]:
# qual_col = ['item_id']
# train_df = pd.get_dummies(train_df, columns=qual_col, prefix=qual_col)

# train_df['item_id'] = train_df.ID.str[0:6]

In [15]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))

predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE")

predictor.fit(train_data=data, 
              time_limit=3600,
              presets="high_quality",
              random_seed=42)

predictor.refit_full()

pred = predictor.predict(data)

# 결과 처리 및 제출 파일 생성
# ...


================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: high_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'high_quality',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': 3600,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231115_144836/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)',

In [16]:
pred = predictor.predict(data)

Global seed set to 123
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [17]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('./submission/with_gg_submission2.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3266.457607
1,TG_A_J_20230305,76.012868
2,TG_A_J_20230306,3264.108216
3,TG_A_J_20230307,3236.527574
4,TG_A_J_20230308,3217.665669
...,...,...
1087,RD_F_J_20230327,582.979526
1088,RD_F_J_20230328,580.387480
1089,RD_F_J_20230329,575.994525
1090,RD_F_J_20230330,568.118361


In [18]:
result = pd.DataFrame()
result['classification'] = predictions_classfication['prediction_label']
result['regression'] = submission['answer']
result.head()

,classification,regression
0,0,3266.457607
1,0,76.012868
2,0,3264.108216
3,0,3236.527574
4,1,3217.665669


In [19]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

In [20]:
result['regression']

0        3266.457607
1           0.000000
2        3264.108216
3        3236.527574
4        3217.665669
            ...     
59392            NaN
59393            NaN
59394            NaN
59395            NaN
59396       0.000000
Name: regression, Length: 59397, dtype: float64

In [21]:
result1 = pd.DataFrame()
result1['ID'] = test['ID']
result1['answer'] = result['regression']

result1.to_csv('./submission/result_anomaly_remove_submission_1115.csv', index=False)

In [22]:
result1

,ID,answer
0,TG_A_J_20230304,3266.457607
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3264.108216
3,TG_A_J_20230307,3236.527574
4,TG_A_J_20230308,3217.665669
...,...,...
1087,RD_F_J_20230327,582.979526
1088,RD_F_J_20230328,580.387480
1089,RD_F_J_20230329,575.994525
1090,RD_F_J_20230330,568.118361
